In [1]:
import sys, os
current_ipynb_path = os.getcwd()

print(current_ipynb_path)
sys.path.insert(0, "../../")

import pandas as pd
import tensorflow as tf

/home/web_server/antispam/project/liuxinyu/projects/tf-library/demo


In [2]:
from rcalgo.tf.utils.tokenizers import CharTokenizer
from rcalgo.tf.models.classifier.text_classifier import TransformerClassifier

In [3]:
config = {
    "learning_rate": 0.0001,

    "num_classes": 2,
    "max_seq_len": 100,
    "embedding_size": 256,
    "num_layers": 2,
    "conv_num_filters": 256,
    "cluster_spec_json" : "./multiworker_training_demo/cluster_config.json",

    "model_output_dir": "./saved_models",
    "model_name": "transformer"
}

In [4]:
# 1. load data
data_dir = "/home/web_server/antispam/project/datasets/female_vulgar_small/"
train_data = pd.read_csv(data_dir + 'train.csv')
test_data = pd.read_csv(data_dir + 'test.csv')
# count word dict
tokenizer = CharTokenizer.build_from_corpus(train_data['text'], freq_threshold=2)
word_dict = tokenizer.word2idx

In [5]:
# 2. create model
model = TransformerClassifier(config, word_dict, name="default", distribute="multiworker_tf_distribute")
model.build_model()

INFO:tensorflow:Single-worker CollectiveAllReduceStrategy with local_devices = ('/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7'), communication = CollectiveCommunication.AUTO
INFO:tensorflow:Multi-worker CollectiveAllReduceStrategy with cluster_spec = {'chief': ['10.46.135.55:18465'], 'worker': ['10.46.131.57:22880']}, task_type = 'chief', task_id = 0, num_workers = 2, local_devices = ('/job:chief/task:0/device:GPU:0', '/job:chief/task:0/device:GPU:1', '/job:chief/task:0/device:GPU:2', '/job:chief/task:0/device:GPU:3', '/job:chief/task:0/device:GPU:4', '/job:chief/task:0/device:GPU:5', '/job:chief/task:0/device:GPU:6', '/job:chief/task:0/device:GPU:7'), communication = CollectiveCommunication.AUTO
INFO:tensorflow:Single-worker CollectiveAllReduceStrategy with local_devices = ('/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/devic

In [ ]:
model.train([train_data['text'], train_data['label']],
            batch_size=512,
            test_size=0.1,
            epochs=1,
            hooks=[])

build_train_graph: reconstructing variables...

build_train_graph: reconstructing variables...
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
build_train_graph (default): building inputs...
task: default, <class 'str'>
[[[<tf.Tensor 'data_batch:0' shape=(?,) dtype=string>, <tf.Tensor 'data_batch:1' shape=(?,) dtype=int32>]], [[<tf.Tensor 'data_batch_1:0' shape=(?,) dtype=string>, <tf.Tensor 'data_batch_1:1' shape=(?,) dtype=int32>]], [[<tf.Tensor 'data_batch_2:0' shape=(?,) dtype=string>, <tf.Tensor 'data_batch_2:1' shape=(?,) dtype=int32>]], [[<tf.Tensor 'data_batch_3:0' shape=(?,) dtype=string>, <tf.Tensor 'data_batch_3:1' shape=(?,) dtype=int32>]], [[<tf.Tensor 'data_batch_4:0' shape=(?,) dtype=string>, <tf.Tensor 'data_batch_4:1' shape=(?,) dtype=int32>]], [[<tf.Tensor 'data_batch_5:0' shape=(?,) dty

replica_id:6, data_batch:[[<tf.Tensor 'data_batch_6:0' shape=(?,) dtype=string>, <tf.Tensor 'data_batch_6:1' shape=(?,) dtype=int32>]]
replica_id:7, data_batch:[[<tf.Tensor 'data_batch_7:0' shape=(?,) dtype=string>, <tf.Tensor 'data_batch_7:1' shape=(?,) dtype=int32>]]
replica_id:7, data_batch:[[<tf.Tensor 'data_batch_7:0' shape=(?,) dtype=string>, <tf.Tensor 'data_batch_7:1' shape=(?,) dtype=int32>]]
INFO:tensorflow:Collective batch_all_reduce: 36 all-reduces, num_workers = 2
INFO:tensorflow:Collective batch_all_reduce: 36 all-reduces, num_workers = 2
build_train_graph (default): aggregating outputs...
train_ops: [PerReplica:{
  0 /job:worker/replica:0/task:0/device:GPU:0: <tf.Tensor 'Adam_default/Identity:0' shape=() dtype=int64>,
  1 /job:worker/replica:0/task:0/device:GPU:1: <tf.Tensor 'Adam_default/Identity_1:0' shape=() dtype=int64>,
  2 /job:worker/replica:0/task:0/device:GPU:2: <tf.Tensor 'Adam_default/Identity_2:0' shape=() dtype=int64>,
  3 /job:worker/replica:0/task:0/device

In [ ]:
input_list = [train_data['text'], train_data['label']]

In [ ]:
dataset_size = len(input_list[0])

In [ ]:
dataset_size%256

In [ ]:
model.trainer.graph.get_collection('input_iter')